In [10]:
import csv
import pandas as pd
from tqdm import tqdm
from requests.compat import urljoin
import requests
import glob
import os
import re
from bs4 import BeautifulSoup

In [11]:
import spacy
nlp = spacy.load("en_core_sci_sm")
# nlp = spacy.load("en_core_web_sm")

In [12]:
AN_RS_PMCIDS_df =  pd.read_csv('../data/accession_data.csv')

/tmp/ipykernel_132314/2228744081.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  AN_RS_PMCIDS_df =  pd.read_csv('../data/accession_data.csv')


In [13]:
AN_RS_PMCIDS_df.columns

Index(['accession_type', 'source', 'extid', 'pmcid'], dtype='object')

In [14]:
AN_RS_PMCIDS_df = AN_RS_PMCIDS_df[['accession_type', 'pmcid']]
AN_RS_PMCIDS_df.head()

,accession_type,pmcid
0,alphafold,PMC10550955
1,alphafold,PMC10526293
2,alphafold,PMC10508174
3,alphafold,PMC10068818
4,alphafold,PMC10518620


In [16]:
size_ = 5

grouped = AN_RS_PMCIDS_df.groupby('accession_type')

# Empty dictionary to store the separate dataframes
dfs = {}

# Loop through each group
for name, group in grouped:
    # Select up to 100 pmcids from each group
    selected = group.head(size_)
    # Store the resulting DataFrame in the dictionary
    dfs[name] = selected
    
combined_df = pd.concat(dfs.values())
combined_df_clean = combined_df.dropna()



In [17]:
combined_df_clean['accession_type'].unique()

array(['alphafold', 'arrayexpress', 'bia', 'biomodels', 'bioproject',
       'biosamples', 'biostudies', 'brenda', 'cath', 'cellosaurus',
       'chebi', 'chembl', 'complexportal', 'dbgap', 'doi', 'ebisc', 'efo',
       'ega', 'emdb', 'empiar', 'ena', 'ensembl', 'eudract', 'gca', 'geo',
       'gisaid', 'go', 'hgnc', 'hipsci', 'hpa', 'igsr', 'intact',
       'interpro', 'metabolights', 'metagenomics', 'mint', 'nct', 'omim',
       'orphadata', 'pdb', 'pfam', 'prode', 'reactome', 'refseq',
       'refsnp', 'rfam', 'rhea', 'rnacentral', 'rrid', 'treefam',
       'uniparc', 'uniprot'], dtype=object)

In [18]:
combined_df_clean.sample(n=10)

,accession_type,pmcid
657684,metabolights,PMC10051505
9596,bioproject,PMC5525643
9192,biomodels,PMC4360671
71002,dbgap,PMC5120758
606017,go,PMC10404263
70934,complexportal,PMC3026084
974803,orphadata,PMC9335361
630219,gisaid,PMC8560824
635640,hpa,PMC9907840
2634333,rnacentral,PMC8190129


In [19]:
pmcids = combined_df_clean['pmcid'].tolist()

In [20]:
import requests
from urllib.parse import urljoin
import csv
from tqdm import tqdm

def get_Json_through_PMCIDs(pmcids, type_):
    base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
    pmcid_query = ','.join([f"PMC%3A{id[3:]}" for id in pmcids])  # Prepare PMCIDs for the query
    article_url = urljoin(base_url, f"annotationsByArticleIds?articleIds={pmcid_query}&type={type_}&format=JSON")
    r = requests.get(article_url)

    if r.status_code == 200:
        return r
    else:
        return False

def get_epmc_annotations_to_file(PMCids):
    with open('../data/AN_RS_annotations_from_api.csv', 'w', newline='\n') as f1:
        test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        # tqdm is now tracking the overall progress
        for i in tqdm(range(0, len(PMCids), 8)):
            batch_ids = PMCids[i:i+8]

            for ann_type in ['Accession Numbers', 'Resources']:
                if ann_type == 'Accession Numbers':
                    ent_type = 'AN'
                elif ann_type == 'Resources':
                    ent_type = 'RS'

                json_annotations = get_Json_through_PMCIDs(batch_ids, ann_type)
                if json_annotations:
                    json_results = json_annotations.json()
                    for article in json_results:
                        try:
                            pmc_id = article['pmcid']
                            for each_annotation in article['annotations']:
                                try:
                                    exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
                                except KeyError:
                                    exact = each_annotation.get('exact', '')
                                token = each_annotation['tags'][0]['name']
                                row = [pmc_id, exact, token, ent_type]
                                test_writer.writerow(row)
                        except (IndexError, KeyError):
                            pass


In [21]:
get_epmc_annotations_to_file(pmcids)

100%|███████████████████████████████████████████| 32/32 [00:37<00:00,  1.17s/it]


In [22]:
annotations_df = pd.read_csv('../data/AN_RS_annotations_from_api.csv', sep='\t', names=['pmcid', 'partial_sentence', 'token', 'ner'])
annotations_df.sample(n=10)

,pmcid,partial_sentence,token,ner
80311,PMC2954584,617105,617105,AN
128247,PMC6909262,13snp_rs6548616_0w_rrs6548616AIMS793995750.3290.1,rs6548616,AN
56556,PMC1501050,7242);neurogenesis (GO:0007399);protein bindin...,GO:0007399,AN
30259,PMC2848993,A*9212——TBC63782AB292417M SatakeA*02:113NA*9,AB292417,AN
106273,PMC4309170,2OM5 2OO0 2OO9 2OOA 2OOQ 2OP7 2OPG 2OPU 2OPV,2OOQ,AN
37456,PMC5525643,S1040153SAMEA3733004ERX1298490ERR1226278ITSITS...,ERX1298490,AN
101486,PMC2848993,�DRB1*1401V2JCB15932AB049831DRB1*14:40:01DRB1*14,AB049831,AN
16000,PMC8035907,which is available via NCBI BioSample SAMN1581...,SAMN15816786,AN
49664,PMC10190287,"84687 42,9887,96565 SRR22784686 5,261,38135,12...",SRR22784686,AN
66601,PMC10404263,"iron ion transport|GO:0055076, transition met...",GO:0055076,AN


In [23]:
annotations_df['ner'].unique()

array(['AN', 'RS'], dtype=object)

In [24]:
list(annotations_df[annotations_df['ner']=='RS'].token.unique())

['Gene Ontology',
 'ArrayExpress',
 'Uniprot',
 'UniProt',
 'Protein DataBank',
 'Ensembl',
 'PDB',
 'BioModels',
 'BioStudies',
 '1000 Genomes',
 'Electron Microscopy Public Image Archive',
 'EMPIAR',
 'Electron Microscopy Data Bank',
 'Protein Data Bank',
 'European Nucleotide Archive',
 'ENA',
 'EGA',
 'PDBe',
 'ChEMBL',
 'MetaboLights',
 'Rfam',
 'Protein Data Bank in Europe',
 'PRIDE',
 'European Variation Archive',
 'EVA',
 'BioSamples',
 'Europe PubMed Central',
 'GWAS Catalog',
 'Experimental Factor Ontology',
 'identifiers.org',
 'Swiss-Prot',
 'UniprotKB',
 'HGNC',
 'InterProScan',
 'InterPro',
 'ChEBI',
 'Chemical Entities of Biological Interest',
 'SILVA rRNA database',
 'SILVA',
 'Swissprot',
 'Pfam',
 'CATH',
 'database STRING',
 'BRENDA',
 'Cellosaurus',
 'cellosaurus',
 'Reactome',
 'IntAct',
 'IntAct database',
 'uniprot',
 'STRING database',
 'UniProtKB',
 'STRING',
 'Complex Portal',
 'RNAcentral',
 'molecular interaction database',
 'Expression Atlas',
 'Rhea knowle

In [25]:
# Get unique PMCIDs
import requests
from bs4 import BeautifulSoup
import re

def get_full_text_xml(pmcid):
    segmented_sentences = []
    url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'xml')
        p_tags = soup.find_all('p')
        p_texts = [tag.get_text() for tag in p_tags]
        # Using nlp.pipe for batch processing
        for doc in nlp.pipe(p_texts):
            sentences = [sent.text.strip() for sent in doc.sents]
            segmented_sentences.extend(sentences)
        return segmented_sentences
    else:
        return None

In [26]:
unique_pmcids = annotations_df['pmcid'].unique()

In [27]:
len(unique_pmcids)

241

In [28]:
def find_sentence_with_substring(string_list, substring):
    for text in string_list:
        sentences = re.split(r'(?<=[.!?])\s+', text)
        for sentence in sentences:
            if substring in sentence:
                return sentence
    return None

def process_pmcid(df, pmcid, p_texts):
    sentences_data = {}
    for _, row in df[df['pmcid'] == pmcid].iterrows():
        sentence = find_sentence_with_substring(p_texts, row['partial_sentence'])
        if sentence:
            if sentence not in sentences_data:
                sentences_data[sentence] = set()

            sentences_data[sentence].add((row['token'], row['ner']))

    return [[pmcid, sentence, list(ner_tags)] for sentence, ner_tags in sentences_data.items()]

final_data = []

for pmcid in tqdm(unique_pmcids):
    p_texts = get_full_text_xml(pmcid)
    if p_texts:
        processed_data = process_pmcid(annotations_df, pmcid, p_texts)
        final_data.extend(processed_data)

# Convert to DataFrame
final_df = pd.DataFrame(final_data, columns=['pmcid', 'sentence', 'ner'])

# Save as TSV
final_df.to_csv('../data/AN-RS_xxx7.tsv', sep='\t', index=False)

100%|█████████████████████████████████████████| 241/241 [14:21<00:00,  3.57s/it]


In [29]:
final_data[:10]

[['PMC8080698',
  'Table 4RNA-seq or gene-chip data from loss or gain of function of each PcG subunit proteina, b.UnitData IDTarget cell (cell line)Functional alteration (method)EZH1E-GEOD-36288LSK (Lin−Sca-1+c-kit+) cellsLoss of function (KO)E-GEOD-59090CD34+ hematopoietic stem and progenitor cell (HSPC)-derived proerythroblastsLoss of function (shRNA)EZH1/ EZH2E-GEOD-62198MLL-AF9 induced leukemia progenitorLoss of function (inhibitor, UNC1999)EZH2E-GEOD-59090CD34+ HSPC-derived proerythroblastsLoss of function (shRNA)E-GEOD-71870Ovarian tumor (OC8 cell line)Loss of function (shRNA)E-GEOD-71671Monocyte (THP-1 cell line)Loss of function (inhibitor, GSK126)E-MTAB-3227Gastric cancer (MKN45 cell line)Loss of function (siRNA)E-GEOD-82072Megakaryocyte-erythrocyte precursorLoss of function (cKO; scl-cre)E-GEOD-82073Long-term-hematopoietic stem cell (LT-HSC)Loss of function (cKO; scl-cre)E-MTAB-2893Chronic myeloid leukemia CD34+ cellsLoss of function (inhibitor, GSK343)E-MTAB-3552Nonchronic my

In [49]:
xx

''

In [52]:
response.content

b'<!DOCTYPE article PUBLIC "-//NLM//DTD JATS (Z39.96) Journal Archiving and Interchange DTD with MathML3 v1.2 20190208//EN" "JATS-archivearticle1-mathml3.dtd"> \n<article xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:ali="http://www.niso.org/schemas/ali/1.0/" article-type="review-article"><?properties open_access?><?DTDIdentifier.IdentifierValue -//Springer-Verlag//DTD A++ V2.4//EN?><?DTDIdentifier.IdentifierType public?><?SourceDTD.DTDName A++V2.4.dtd?><?SourceDTD.Version 2.4?><?ConverterInfo.XSLTName springer2nlmx2.xsl?><?ConverterInfo.Version 1?><front><journal-meta><journal-id journal-id-type="nlm-ta">Exp Mol Med</journal-id><journal-id journal-id-type="iso-abbrev">Exp Mol Med</journal-id><journal-title-group><journal-title>Experimental &#x00026; Molecular Medicine</journal-title></journal-title-group><issn pub-type="ppub">1226-3613</issn><issn pub-type="epub">2092-6413</issn><publisher><publisher-name>Nature Publishing Group UK</pu

In [19]:
import ast

def calculate_spans(sentence, entities):
    spans = []
    for entity, etype in entities:
        start = sentence.find(entity)
        if start != -1:
            end = start + len(entity)
            spans.append([start, end, entity, etype])  # List instead of tuple
    return spans

def resolve_overlaps(spans):
    spans.sort(key=lambda x: (x[1] - x[0]), reverse=True)  # Sort by span length
    resolved = []
    for span in spans:
        overlap = False
        for r in resolved:
            if not (span[1] <= r[0] or span[0] >= r[1]):  # Check for overlap
                overlap = True
                break
        if not overlap:
            resolved.append(span)
    return resolved



In [20]:
df = pd.read_csv('../data/xxx7.tsv', sep = '\t')
df['new_ner'] = df.apply(lambda row: resolve_overlaps(calculate_spans(row['sentence'], ast.literal_eval(row['ner']))), axis=1)
df.drop('ner', axis=1, inplace=True)
df.rename(columns={'new_ner': 'ner'}, inplace=True)
df.to_csv('../data/xxx6.tsv', sep = '\t', index=False)

In [23]:
# Rename entity names in file2.csv as per the provided mapping
entity_rename_map = {
    'Gene Ontology': 'GO',
    'Experimental Methods': 'EM', 
#     'Accession Numbers': 'AN',
#     'Resources': 'RS',
    'Chemicals': 'CD'
}

file2_df = pd.read_csv('../data/xxx6.tsv', sep = '\t')
# Applying the renaming
file2_df['ner'] = file2_df['ner'].apply(lambda x: eval(x))
for row in file2_df['ner']:
    for entity in row:
        entity[3] = entity_rename_map.get(entity[3], entity[3])

# Checking the changes
file2_df.head()


,pmcid,sentence,ner
0,PMC4792959,A subunit of the oligosaccharyltransferase com...,"[[17, 50, oligosaccharyltransferase complex, GO]]"
1,PMC4792959,"During plant pollen tube (PT) reception, gamet...","[[13, 24, pollen tube, GO]]"
2,PMC4792959,ARU encodes the OST3/6 subunit of the oligosac...,"[[38, 71, oligosaccharyltransferase complex, GO]]"
3,PMC4792959,Our results suggest that glycosylation pattern...,"[[25, 38, glycosylation, EM]]"
4,PMC4792959,show that ARTUMES regulates pollen tube recogn...,"[[28, 39, pollen tube, GO]]"


In [24]:
# Correcting the code to work with the 'ners' column, use sets to remove duplicates, 
# and applying specific handling for 'AN' type

# Adjusting the safe_eval_list function to check if the input is a string
def safe_eval_list(str_list):
    if isinstance(str_list, str):
        try:
            return eval(str_list)
        except SyntaxError:
            return []
    return str_list

# Applying the adjusted renaming and extraction
file2_df['ner'] = file2_df['ner'].apply(safe_eval_list)
for row in file2_df['ner']:
    for entity in row:
        entity[3] = entity_rename_map.get(entity[3], entity[3])

        
def replace_substrings(input_str):
    substrings_to_replace = ['*/', '18O ', '% ', 'μm', 'its ', 'are ',  'successful ', 'III ', '6 ', 
                             'their ', 'μm ', 'between ', 'some ', 'through ', 'mL ', 'while ','s layer', '50 ', 
                             'vs ', 'induces ', 'p53 ', 'promotes ', 'within ', 'precise ', 'nitrogen ', '—0', 
                             'other ', 'created ', 'basement ', 'best ', 'study ', 'than ', '94°C,', 'followed ', 
                             '°CAbbreviations:', '7500', 'simultaneous', 'represent', 'II ', '15 ', '10× ', 
                             'initiation ', 'all ', 'toward ', 'improve ', 'whose ', 
                             'influence ', 'convey', 'and/or ', 'acid ', 
                             'analyzed ', 'thus ', 'per ', 'affect ', 'associated ', 'increased' , 
                             'releases ', 'kill', 'Nonetheless, ', 
                            '“', 'against ', 'additional ', 'late ', 'reduced ', 'Distribution ', 
'giving ', 'maintain ', 'nighttime ', 'increase ', 'during ', '30 ' 'HIV ',
'whether ', '- ', 'use ', 'Three ', 'Caused', 'block ', 'IL-8 ', 'No ', 'create ', '”'
'Il-21 ', 'what ', 'base ', 'paid ', 'then ', 'without ', 'where ', '16100 ', '2 ', '325,000 ', 'underlying ', 'related ']
    
    for substring in substrings_to_replace:
        input_str = input_str.replace(substring, '')
    
    return input_str


# Extracting dictionary collection of set with AN, EM, RS, and GO
extracted_entities = {'AN': set(), 'EM': set(), 'RS': set(), 'GO': set(), 'CD': set()}
for row in file2_df['ner']:
    for entity in row:
        entity_type = entity[3]
        if entity_type in extracted_entities:
            # For 'AN', split with a space and take the last term
            if entity_type == 'AN':
                extracted_entities[entity_type].add(entity[2].split()[-1])
            elif entity_type == 'RS' and 'database' in entity[2]:
                extracted_entities[entity_type].add(entity[2].replace('database','').strip())
            elif entity_type == 'GO' or entity_type == 'EM':
                if 'membrane' in entity[2].lower() or 'behaviour' in entity[2].lower():
                    extracted_entities[entity_type].add(entity[2].split()[-1])
                else:
                    replac_str = replace_substrings(entity[2]).strip()
                    if replac_str and len(replac_str)>1:
                        extracted_entities[entity_type].add(replac_str)
            else:
                extracted_entities[entity_type].add(entity[2])


# Converting sets back to lists for easier viewing/usage
extracted_entities = {k: list(v) for k, v in extracted_entities.items()}

# extracted_entities



In [22]:
# extracted_entities['RS'].extend(['NHGRI', 'GEO', 'Sequence Read Archive', 'SRA', 'KEGG', 
#                                  'Kyoto Encyclopedia of Genes and Genomes', 'IntAct62',
#                                  'MINT63', 'AtPIN36', 'AtPID37', 'PAIR38', 'APID39', 'PRIN9', 'PHISTO', 'TAIR',
#                                 'InParanoid', 'HPRD', 'Medline', 'EMBASE', 'PsycINFO', 'EconLit', 'HEED',
#                                  'CRD', 'Health Economic Evaluations Database', 'HEED', 'NHS Economic Evaluation Database',
#                                 'NHS EED', 'Database of Abstracts of Reviews of Effects', 'DARE', 'PROSITE',
#                                 'CINAHL', 'Cochrane', 'ARAMEMNON', 'TrEMBL', 'PhosphoSitePlus', 
#                                  'UniProtKB/TrEMBL', 'SwissProt', 'miRWalk',
#                                  'DAVID', 'SEER', 'Genebank','MSigDB', 'MGI', 'PsyGeNET', 'SFARI', 'DisGeNET',
#                                 'GeoSymbio', 'TIGR', 'GenBank', 'Swiss-prot' ])

In [23]:
# new_accessions = list(set(['FN813615', 'FN813614', 'AM042701', 'AM042698', 'AM042697', 'AM042699', 'FN823053', 
#                   'FN995351', 'KY355383', 'ENSDARG000000', 'XP_694951', 'XP_694893', 'XP_686111', 
#                   'XP_686816', 'XP_689021', 'XP_693637', 'XP_694145', 'XP_693652', 'XP_691478', 
#                   'XP_689023', 'XP_698503', 'XP_690676', 'XP_689954', 'XP_693868', 'XP_688350', 
#                   'XP_697815', 'XP_687258', 'XP_690327', 'XP_687591', 'XP_691896', 'XP_689864', 
#                   'XP_690860', 'XP_694605', 'XP_694871', 'XP_696432', 'XP_698814', 'XP_699128', 
#                   'XP_697396', 'ENSGALG00000011535', 'XM_414969', 'XM_414012', 'XM_414012', 'Q5F3J4', 
#                   'ENSXETG00000022012', 'GENSCAN00000015124', 'AC157698', 'ENSXETG00000021586', 
#                   'scaffold_279.46c', 'scaffold_1247.5', 'scaffold_3599.1', 'scaffold_626.23', 
#                   'scaffold_279.46c', 'scaffold_626.24', 'scaffold_626.25', 'scaffold_279.46c', 
#                   'scaffold_279.46c', 'scaffold_111.48', 'ENSXETG00000018490', 'NEWSINFRUG00000144256', 
#                   'NEWSINFRUG00000134206', 'NEWSINFRUG00000138086', 'GENSCANSLICE00000000561', 
#                   'NEWSINFRUG00000148012', 'TC326097', 'TC353741', 'TC343155', 'EST487', 'MF468321', 
#                   'MF468322', 'MF468319', 'MF468318', 'MF468291', 'MF468293', 'MF468299', 'MF468290', 
#                   'MF468308', 'MF468291', 'MF468293', 'MF468294', 'MF468298', 'MF468309', 'MF468314', 
#                   'MF468295', 'MF468297', 'MF468325', 'MF468328', 'AAL56428', 'AAL56429', 'AAB05820', 
#                   'U39815', 'AF126830 ', 'AAL56430', 'AAL58703', 'AAL58704 ', 'rs4179590', 'rs4179590', 
#                   'rs4179589', 'rs51960690', 'rs50972129', 'rs4179517', 'rs4179515', 'rs48139623', 
#                   'rs32704753', 'rs4179510', 'rs4179504', 'rs4179702', 'rs31418354', 'rs4179917', 
#                   'rs4179913', 'rs45694900', 'rs4179908', 'Q1051R', 'rs4179907', 'rs4179728', 
#                   'rs4179729', 'rs4179745', 'ENSMUSSNP328538', 'rs32712110', 'rs4179843', 'rs4179872', 
#                   'rs31418356', 'rs32712642', 'PRJNA375720', 'AY157059', 'AJ000520', 'BA000042', 
#                   'ABE84009', 'ABE86610', 'AE017283', 'NM116255', 'CP000323', 'CP000653', 'AJ316032', 
#                   'JQ804931', 'GSE97544', 'PRJEB9815', 'GSM1334015', 'GSM1334016', 'GSM1334017', 
#                   'GSM1334018', 'GSM1334019', 'GSM1334020', 'KY355383', 'Q7RTR2', 'Q9Y239', 'Q7RTR2', 
#                   'Q86WI3', 'Q7RTR3', 'Q9C000', 'Q9NX02', 'Q96P20', 'Q96MN2', 'P59047', 'P59044', 
#                   'Q32MH8', 'Q86W28', 'Q86W27', 'Q86W26', 'P59045', 'P59046', 'Q86W25', 'Q86W24', 
#                   'Q9NPP4', 'Q13075', 'P33076', 'KT184354', 'DQ136185', 'AM492684a', 'EU339226', 
#                   'EU339209', 'EU339214', 'EU339217', 'Ynys1', 'AF387302a', 'EU339223', 'EU339216', 
#                   'WJ68', 'AY455805a', 'EU339224', 'EU339213', 'AY950676a', 'EU339231', 'EU304260a', 
#                   'EU339222', 'AY455808a', 'EU339230', 'EU339215', 'GSE24189', 'KJ598498.1', 'GSE9210', 
#                   'KF309066', 'MH238461', 'GSE43316', 'AY335912', 'AM902709', 'PRJNA236112', 'AM492684a', 
#                   'EU339226', 'EU339209', 'EU339214', 'EU339217', 'Ynys1', 'AF387302a', 'EU339223', 
#                   'EU339216', 'WJ68', 'AY455805a', 'EU339224', 'EU339213', 'AY950676a', 'EU339231', 
#                   'EU304260a', 'EU339222', 'AY455808a', 'EU339230', 'EU339215', 'EU792508', 'ACF19852', 
#                   'EU792509', 'ACF19853', 'EU792510', 'ACF19854', 'EU792511', 'ACF19855', 'AAC32826', 
#                   'ABK76312', 'AFJ95132', 'AAC48340', 'AF143817', 'AF041023', 'AB009305', 'AF126830', 
#                   'AAL56430', 'AAL58703', 'AAL58704', 'AAL56428', 'AAL56429', 'AAB07368', 'AAB05820', 
#                   'U39815', 'AF167707', 'AF167708', 'AF167709', 'AF167710', 'CAC28360', 'U25057', 
#                   'BAA14015', 'AAD01628', 'AAB26196', 'P54190', 'U29761', 'AAB53231', 'ACJ64718', 
#                   'AAB00227', 'SRP049689', 'EG10787', 'EG11221', 'EG12144', 'EG10379', 'EG10283', 
#                   'EG11448', 'EG10688', 'EG10489', 'EG10283', 'EG20173', 'EG10702', 'KX262676.1', 
#                   'KX352162', 'KF532967', 'NM_000484', 'KX553930', 'KX553934', 'AM117604', 'FN813615', 
#                   'FN813614', 'AM042701', 'AM042698', 'AM042697', 'AM042699', 'FN823053', 'FN995351', 
#                   'AM117602.1', 'NM-138554.1', 'P25297', 'PHO84', 'NP_001337.2', 'NP_003638', 'GSE50635','MF468290', 'SRR4011627', 'SRR4011628', 'HM019534', 'HM020374', 'FN813615',
# 'FN813614', 'AM042701', 'AM042698', 'AM042697', 'AM042699', 'HM019533', 'HM020128', 'HM020130', 'HM020332',
# 'HM202334', 'HM020374', 'COSM43967', 'COSM1640833', 'COSM562645', 'COSM44853', 'COSM99631']))

In [24]:
len(new_accessions)  #previously 299

258

In [25]:
# print(len(extracted_entities['AN']))
# extracted_entities['AN'].extend(new_accessions)
# print(len(extracted_entities['AN']))

158
416


In [26]:
# with open('../data/CD_terms_list.txt', 'w') as file:
#     for term in list(set(extracted_entities['CD'])):
#         file.write(term + '\n')

In [25]:
chem_to_remove = ['uc.134', 'uc','Yorkie','chemical substances','sec','Cuban','8690176','131127258',
'CD','DS','steward','ZF-EST487-R3','-EST487','delta','n13131313','6429','sc-376167','QX-314','Cy3',
'retina','gamma','singular','electron','1474310','1415180','HM5040',
'singular','sec ','Nile Red ','91558 ','KK4824 ','910111213 ','23333435 ','678545556 ','torques ','dump',
'torque','at a p','syber green','Short','Short chain fatty acids','2021222324','11192732','1001', '90th',
'Gamma',  'Tech', 'fade', ' His-', '-pro-', 'mold', 'electton', 'Psi', 'radio', 'Radio', 'Cho',  'Fmoc', 
 'aaa', 'h groups', 'His-', '3At', 'a steroid', 'Cy3', 'Tris buffers', 'mega', 
 'Halogen', 'Ocn', 'an alcohol', 'fmoc', 'lime', '-Trp', '-Pro', '-His', 'Photon', ' suc', 'nape', 
'Texas red', 'Clo', ' pro-', 'polar amino acids', 'nonyl', 'Dapi', 'hydroxyl radicals', 'aromatic amino acid',
  'Foscarnet', 'mannose', ' Cho', 'at a p', '3 ms', '3 at', 'bile salts', 'procure',  'thio',  'an aldehyde',               
  'Dopa', 'limestones', ' sec', ' Pro-', 'Retina', 'e is', 'flash',   'molds', 'Flash', 'radical', 'torques']

In [26]:
filtered_list = [term for term in list(set(extracted_entities['CD'])) if term not in chem_to_remove]
# filtered_list

In [27]:
# with open('../data/CD_terms_list.txt', 'w') as file:
#     for term in list(set(filtered_list)):
#         file.write(term + '\n')

In [28]:
extracted_entities['GO'].remove('part')
# extracted_entities['RS'].remove('HPA')
extracted_entities['CD'] = filtered_list

In [29]:
# import pandas as pd

# def enrich_entities(file_path, new_entity_dict):
#     # Load the CSV file
#     df = pd.read_csv(file_path, names=['sentence', 'ner'])

#     # Function to find if there is an overlap with existing spans or if the term is a substring
#     def is_valid_entity(sentence, new_span, existing_spans, current_entity_type):
#         start_idx, end_idx = new_span
#         # Check for overlap with existing spans and allow 'AN' entities to override 'CD'
#         for span, span_type in existing_spans:
#             if not (end_idx <= span[0] or start_idx >= span[1]):
#                 # Allow 'AN' to override 'CD'
#                 if current_entity_type == 'AN' and span_type == 'CD':
#                     continue
#                 return False
#         # Check if the entity is not part of a larger word (to avoid substrings)
#         if (start_idx > 0 and sentence[start_idx - 1].isalnum()) or (end_idx < len(sentence) and sentence[end_idx].isalnum()):
#             return False
#         return True

#     # Function to add new entity annotations
#     def add_new_entities(sentence, entities, entity_dict):
#         existing_spans = [(tuple(map(int, entity[:2])), entity[3]) for entity in entities if entity != 'None']  # Tuple of span and type
#         for entity_type, terms in entity_dict.items():
#             for term in terms:
#                 start_idx = sentence.find(term)
#                 while start_idx != -1:
#                     end_idx = start_idx + len(term)
#                     new_span = (start_idx, end_idx)
#                     if is_valid_entity(sentence, new_span, existing_spans, entity_type):
#                         entities = [entity for entity in entities if not (entity[0] == start_idx and entity[3] == 'CD')]  # Remove overridden 'CD'
#                         entities.append([start_idx, end_idx, term, entity_type])
#                         existing_spans.append((new_span, entity_type))
#                     start_idx = sentence.find(term, start_idx + 1)
#         return [entity for entity in entities if entity != 'None']  # Exclude 'None' if other entities are present

#     # Enriching the entity annotations
#     for i, row in df.iterrows():
#         sentence = row['sentence']
#         entities = eval(row['ner']) if row['ner'] != "['None']" else ['None']
#         df.at[i, 'ner'] = add_new_entities(sentence, entities, new_entity_dict)

#     return df

import pandas as pd

def enrich_entities(file_path, new_entity_dict, chem_to_remove):
    # Load the CSV file
    df = pd.read_csv(file_path, names=['sentence', 'ner'])

    # Function to find if there is an overlap with existing spans or if the term is a substring
    def is_valid_entity(sentence, new_span, existing_spans, current_entity_type):
        start_idx, end_idx = new_span
        # Check for overlap with existing spans
        for span, span_type in existing_spans:
            if not (end_idx <= span[0] or start_idx >= span[1]):
                return False
        # Check if the entity is not part of a larger word (to avoid substrings)
        if (start_idx > 0 and sentence[start_idx - 1].isalnum()) or (end_idx < len(sentence) and sentence[end_idx].isalnum()):
            return False
        return True

    # Function to add new entity annotations
    def add_new_entities(sentence, entities, entity_dict):
        existing_spans = [(tuple(map(int, entity[:2])), entity[3]) for entity in entities if entity != 'None']  # Tuple of span and type
        for entity_type, terms in entity_dict.items():
            for term in terms:
                start_idx = sentence.find(term)
                while start_idx != -1:
                    end_idx = start_idx + len(term)
                    new_span = (start_idx, end_idx)
                    if is_valid_entity(sentence, new_span, existing_spans, entity_type):
                        entities.append([start_idx, end_idx, term, entity_type])
                        existing_spans.append((new_span, entity_type))
                    start_idx = sentence.find(term, start_idx + 1)

        # Remove 'CD' entities if they are in chem_to_remove list
        entities = [entity for entity in entities if not (entity[2] in chem_to_remove and entity[3] == 'CD')]
        return [entity for entity in entities if entity != 'None']  # Exclude 'None' if other entities are present

    # Enriching the entity annotations
    for i, row in df.iterrows():
        sentence = row['sentence']
        entities = eval(row['ner']) if row['ner'] != "['None']" else ['None']
        df.at[i, 'ner'] = add_new_entities(sentence, entities, new_entity_dict)

    return df


In [30]:
def find_sub_span(sub_span_range, full_spans_range):
    # Check if a sub span is present in full span
    if sub_span_range[0] in range(full_spans_range[0], full_spans_range[1]):
        return sub_span_range

def convert2IOB(text_data, ner_tags):
    if 'None' in ner_tags:
        # Split text into words and return with 'O' tags if there are no NER tags
        split_text = tokenizer.tokenize(text_data)
        return zip(split_text, ['O'] * len(split_text))

    # Tokenize text and create a list of token spans
    tokens = tokenizer.tokenize(text_data)
    current_pos = 0
    token_spans = []
    for token in tokens:
        start_pos = text_data.find(token, current_pos)
        end_pos = start_pos + len(token)
        token_spans.append((start_pos, end_pos))
        current_pos = end_pos

    # Initialize IOB tags as 'O'
    iob_tags = ['O'] * len(tokens)

    # Process each NER tag
    for start, end, entity, entity_type in sorted(ner_tags, key=lambda x: x[0]):
        for i, span in enumerate(token_spans):
            if start <= span[0] < end:
                if start == span[0]:
                    # Beginning of entity
                    iob_tags[i] = 'B-' + entity_type
                else:
                    # Inside of entity
                    iob_tags[i] = 'I-' + entity_type

    return zip(tokens, iob_tags)



In [31]:
import csv
import pathlib
from nltk.tokenize import WordPunctTokenizer, wordpunct_tokenize
tokenizer = WordPunctTokenizer()

def convert_to_IOB_format(data_file, output_folder,filename):
    # Read the CSV file
    df = pd.read_csv(data_file)

    # Define the output path
    result_path = os.path.join(output_folder, filename)
    pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    with open(result_path, 'w', newline='\n') as f1:
        train_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
        
        for index, row in df.iterrows():
            text = row['sentence']
            ner_tags = eval(row['ner']) if row['ner'] != "['None']" else ['None']
            tagged_tokens = convert2IOB(text, ner_tags)
            for each_token in tagged_tokens:
                train_writer.writerow(list(each_token))
            train_writer.writerow('')

output_folder = '../data/enriched_IOB/'

In [34]:
# Example usage
filename = 'test.csv'

file_path = '../data/CD_GP_DS_OG_test/clean_CD_GP_DS_OG_'+filename
new_df = enrich_entities(file_path, extracted_entities, chem_to_remove)
new_df.to_csv(file_path.replace('clean_','enriched_2_'), index=False)

data_file = '../data/CD_GP_DS_OG_test/enriched_1_CD_GP_DS_OG_'+filename
convert_to_IOB_format(data_file, output_folder,filename)

In [2]:
# This is a XML aware sentenciser for JATS DTD. Sentence splitting rules are based on SSSplit.
# Developed by Shyamasree Saha, 2021

import argparse
import sys
import regex as re
import io
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm

# sys.setrecursionlimit(2500)
# print(sys.getrecursionlimit())
## defining all the patterns
bad_white_space = "(\t|\r|\n)"
space_btw_tags = "> <"
single_fig_graphic = "<FIGURE/>|<graphic.*?/>"  # replace with ""
fig_stop = "\\.<fig "  # replace with ". <fig "
table_wrap_stop = "\\.<table-wrap "  # replace with ". <table-wrap "
allowedAttrChars = "[\\w-_\u2013\\.\\(\\)\\[\\]]"
refSCIgeneral = "(?:(?:(?:\\d+[,\u2013])*\\d+)?(?:</italic>)?<REF(?:(?: ID=\"(?:\\w-)?\\w+?(?: \\w+?)*\")|(?: " \
                "REFID=\"\\w+?\")|(?: text=\"(?:refs?\\.(?: )?)?(?:(?:\\d+[,�\u2013\u002D])*\\d+)?\\w*?\")|(?: " \
                "TYPE=\"\\w+?\"))*(?: ?/>|>(?:(?:refs?\\.(?: )?)?(?:(?:(?:\\d+(?:<italic>|<i>)?\\w?(?:</italic>|</i>)?)|(?:(" \
                "?:<italic>|<i>)?\\w(?:</italic>|</i>)?[-,\u2013])|(?:(?:<italic>|<i>)?\\w(?:</italic>|</i>)?[-,\u2013]))*(?:(?:\\d+(" \
                "?:<italic>|<i>)?\\w?(?:</italic>|</i>)?)|(?:(?:<italic>|<i>)?\\w?(?:</italic>|</i>)?)))?</REF>))|(?:<xref(?: " \
                "id=\"(?:\\w-)?\\w+?(?: \\w+?)*\")?(?: ref-type=\"(" \
                "?:aff|app|author-notes|bibr|boxed-text|chem|contrib|corresp|disp-formula|fig|fn|kwd|list|plate|ref" \
                "|scheme|sec|statement|supplementary-material|table|table-fn|other)\")?(?: rid=\"(?:[" \
                "-\u2013\\w])*\")?>(?:(?:<sup>)?[-\u2013,\\d]+(?:</sup>))</xref>))"
# refFootnote = "<SUP TYPE=\"FOOTNOTE_MARKER\" ID=\"[^\"]+?\"/>"
atLeastOneRefSCI = "(" + refSCIgeneral + "+" + "\\)?|\\))"  # there may be a bracket after the
# reference
capturePunctuation = "(\\.|\\?|(?<!\\(\\w{1,15})\\!)"
abbreviations = '((\\(|\\[| |>|^)(al|Am|Angew|approx|Biochim|Biophys|ca|cf|Chem|Co|conc|Dr|Drs|Corp|Ed|no|No|Sci|Nat' \
                '|Rep|e\\.g|eg|p\\.p\\.m|Engl|eq|eqns?|exp|Rs|Figs?|Labs?|Dr|etc|Calc|i\\.e|ie|Inc|Int|Lett|Ltd|p|p\\.a' \
                '|Phys|Prof|prot|refs?|Rev|sect|st|vs|(?<!(?:</SB>|(?:\\d ?(<italic>?))|<italic>))(?:(?:(?:[A-Z]|[' \
                'a-z])?\\.) ?[a-z])|(?<!(?:</SB>|(?:\\d ?(<italic>?))|<italic>))(?:(?:(?:[A-Z]|[a-z])\\.)? ?(?:(?:[A-Z]|[' \
                'a-z])\\.)? ?[A-Z])|(?<!(?:</SB>|(?:\\d ?(<italic>?))|<italic>))(?:(?:(?:[A-Z]|[a-z])\\.)? ?(?:(?:[A-Z]|[' \
                'a-z])\\.) ?[a-z])|(?<=(?:(?<!(?:</SB>|(?:\\d)))(?:(?: |\\(|^)<italic>)))(?: ?\\w{1,10})| ?\\. ?\\. ?|\\())'
# moving all references inside the punctuation so that the sentence splitting is easier
refSentence = "(.*?)" + capturePunctuation + atLeastOneRefSCI + "(\\s?(?:</P>)?)"
abbrevs = ".*?(" + abbreviations + "(?:|</i>|</italic>)?)" + "$"
capitals = "[A-Z0-9]"  # caps and numbers may begin a sentence
punctuation = "(?:\\.|\\?|(?<!\\(\\w{1,15})\\!)"  # .?!     "(?:\\.|\\?|(?<!\\(\\w{1,15}))\\!|\\:(?=<list.+>))"
optPunctuation = punctuation + "??"
endEquation = "</EQN>"
# String endPara1 = "(</P>|</ABSTRACT>)"
endPara = "(</p>|</list>)"
beginPara = "<p>"
# notFollowFigTab = "(?!(<fig)|(<table-wrap))";
optStartQuote = "['\"\u201C]?"
optCloseQuote = "['\"\u201D]?"
optReferenceSCI = refSCIgeneral + "*"
beginFirstSentence = "^"
endLastSentence = "$"
openHeader = "(<HEADER/>|<label>)"
wholeHeader = "((<BODY>)?(<DIV( DEPTH=\"\\d+\")?>)?(<HEADER( HEADER_MARKER=\"" + allowedAttrChars + \
              "+?\")?>.*?</HEADER>|<HEADER/>)|<TITLE>)"
optOpenHeader = openHeader + "?"

eqn = "<EQN( ID=\"" + allowedAttrChars + "+?\")?( TYPE=\"" + allowedAttrChars + "+?\")?>"
xref = "<xref( .+)?>"
listTag = "<list.+?>"
# manyStartTags = "(" + eqn + "|" + xref + "|<BODY>|<DIV( DEPTH=\"\d+\")?>|<P>|<B>|<IT>)*"
manyStartTags = "(" + xref + "|" + listTag + "|<p.+?>|<license-p>|<title>|<caption>|<sec.+?>|<disp-quote>|<supplementary-material.+?>|<boxed-text.+?>|<list list-type=\"\\w{2,20}\">|<list-item>|<article-title>|<abstract.+>|<AbstractText.+>|<statement>|<def>|<fig .+>)*"  # shs listTag on 13/10/09, <ABSTRACT>|<abstract.+> on 09/11/09
optEndTags = "(</label>|</boxed-text>|</list>|</list-item>|</statement>|</copyright-statement>)?"
# <list.+?> added to optEndTags1
optEndTags1 = "(</caption>|</boxed-text>|<list.+?>|</list>|</list-item>|</statement" \
              ">|</inline-supplementary-material>|<inline-supplementary-material/>|</related-article>|</related" \
              "-object>|</address>|</alternatives>|</array>|</boxed-text>|</chem-struct-wrap>|</fig>|</fig-group" \
              ">|<graphic.+?/>|</media>|</preformat>|</supplementary-material>|</table-wrap-foot>|</table-wrap" \
              ">|</table-wrap-group>|</disp-formula>|</disp-formula-group>|</element-citation>|</mixed-citation" \
              ">|</nlm-citation>|</bold>|</italic>|</monospace>|</overline>|</overline-start>|</overline-end>|</roman" \
              ">|</sans-serif>|</sc>|</strike>|</underline>|</underline-start>|</underline-end>|</award-id>|</funding" \
              "-source>|</open-access>|</chem-struct>|<inline-formula/>|<inline-graphic/> " \
              "|</private-char>|</def-list>|</list>|</tex-math>|</mml:math>|</abbrev>|</milestone-end>|</milestone" \
              "-start>|</named-content>|</styled-content>|</ack> " \
              "|</disp-quote>|</speech>|</statement>|</verse-group>|</fn>|</target>|</xref>|</ref>|</sub>|</sup>" \
              ">|</def>)*"
# </P> added on 23-11-10 by shs
endTags = "(</licence-p>|</italic>|</sup>|</bold>|</article-title>|</title>|</abstract>|<disp-formula>|</disp-quote>|</supplementary-material>|</boxed-text>|</list>|</list-item>|</statement>)"
manyEndTags = endTags + "*"
# String sentenceTerminator = "(?>" +endPara+ "|"+ endEquation +"|" + "(?<!(\ |>)refs?)"+punctuation+")"
endParaOrEq = "(" + endPara + "|" + endEquation + ") ?"  # shs notFollowedBy added
formatting = "(<B>|<IT>|<i>|<SP>|<italic>|<BOLD>|<bold>|<sup>|<disp-formula>|<named-content.+?>)"
puncNoAbbrv = "(?<!" + abbreviations + "(</italic>)?)" + punctuation + " "
greekLetters = "[\u0370-\u03FF\u1F00-\u1FFF]"
pAttr = "<p.+?>"
sentenceCommencer = "(?>" + beginPara + "|" + pAttr + "|" + "Fig(s)?\\." + "|" + "fig(s)?\\." + "|" + capitals + "|" + \
                    formatting + "|" + "\\[|\\(|" + greekLetters + "|\u007C)"
equationCommencer = "(" + eqn + ".)"
commencer = "(" + sentenceCommencer + "|" + equationCommencer + ")"
noSpaceReqLookahead = manyStartTags + optOpenHeader + optStartQuote + commencer
nocapsParaLookAhead = "( ?<P>)"
startSentence = manyStartTags + optStartQuote + commencer
# For matching the end of the previous sentence
sentenceFigLookbehind = "(?<=(?<!" + abbreviations + punctuation + ")((" + endParaOrEq + ")|(" + puncNoAbbrv + ")|(" \
                        + optPunctuation + optEndTags + endTags + " ?)))"

# for matching the start of a sentence following a header
headerLookahead = "(?=(?:" + manyStartTags + optOpenHeader + optStartQuote + commencer + "))"
# (Headerstuff | ((normal sentence | firstsentence) sentenceContent, (punctuation|endEquation), optionalEndings,
# lookahead))
# String figLookAhead = "";
Figure = "<fig ?.+?</fig>"
tableWrap = "<table-wrap.+?</table-wrap>|<table-wrap-foot.+?</table-wrap-foot>"
title = "((?:<title/>)|(?:<title.+?</title>))"  # "<title.+?</title>"
# String absBodyLookBehind = "((?:<abstract>)|(?:<abstract.+?>)|(?:<body>))"
secLookBehind = "((?:<sec>)|(?:<sec.+?>))?"  # <sec> can be followed by<body>.
supplimentbehind = "<supplementary-material.+?</supplementary-material>"
refList = "<ref-list.+?</ref-list>"
boxed = "(?:<boxed-text.+?>)(?:<caption>)?"


# def get_blocks_from_xml_string(xml_string, document_flag):
#     sub_file_blocks = []
#     xml = '<?xml version="1.0" encoding="UTF-8"?>'
#     doc_str = '<!DOCTYPE '
#     start_str = '<article '
#     start_str2 = '<articles><article '
#     doc_str_flag = 0

#     lines = xml_string.split('\n')
#     for line in lines:
#         if document_flag == 'f':
#             if line.startswith(start_str) or line.startswith(start_str2):
#                 sub_file_blocks.append(line.replace('^<articles>', ''))
#                 doc_str_flag = 0
#             elif line.startswith(doc_str):
#                 doc_str_flag = 1
#             else:
#                 if doc_str_flag != 1:
#                     sub_file_blocks[-1] += line.strip().replace('</articles>$', '')
#         elif document_flag == 'a':
#             start_str1 = '<article>'
#             start_str2 = '<articles>'
#             if line.startswith(start_str1):
#                 sub_file_blocks.append(line)
#             elif line.startswith(start_str2):
#                 continue
#             else:
#                 if len(sub_file_blocks) > 0:
#                     sub_file_blocks[-1] += line
#         else:
#             print('ERROR: unknown document type :' + document_flag)
#             return []

#     return sub_file_blocks

def get_blocks_from_xml_string(xml_string, document_flag):
    sub_file_blocks = []
    xml = '<?xml version="1.0" encoding="UTF-8"?>'
    doc_str = '<!DOCTYPE '
    start_str = '<article '
    start_str2 = '<articles><article '
    doc_str_flag = 0

    lines = xml_string.split('\n')
    for line in lines:
        if document_flag == 'f':
            if line.startswith(start_str) or line.startswith(start_str2):
                sub_file_blocks.append(line.replace('^<articles>', ''))
                doc_str_flag = 0
            elif line.startswith(doc_str):
                doc_str_flag = 1
            else:
                if doc_str_flag != 1:
                    if sub_file_blocks:
                        sub_file_blocks[-1] += line.strip().replace('</articles>$', '')
        elif document_flag == 'a':
            start_str1 = '<article>'
            start_str2 = '<articles>'
            if line.startswith(start_str1):
                sub_file_blocks.append(line)
            elif line.startswith(start_str2):
                continue
            else:
                if sub_file_blocks:
                    sub_file_blocks[-1] += line
        else:
            print('ERROR: unknown document type :' + document_flag)
            return []

    return sub_file_blocks


def sentence_split(clear_string, sid):
    # clear_string = str(xml_item)
    #print(clear_string)
    # Apply all replace statements
    clear_string = re.sub(bad_white_space, '', clear_string)
    clear_string = re.sub(space_btw_tags, '><', clear_string)
    clear_string = re.sub(single_fig_graphic, '', clear_string)
    clear_string = re.sub(fig_stop, '. <fig ', clear_string)
    clear_string = re.sub(table_wrap_stop, '. <table-wrap ', clear_string)
    clear_string = re.sub("</xref>, <xref", "</xref><xref", clear_string)
    # print(refSentence[740:])
    refSentence_pattern = re.compile(refSentence, flags=0)
    matches = re.finditer(refSentence_pattern, clear_string)
    # print(matches)
    swapped_string = ""
    ref_last = None
    for refm in matches:
        a = refm.group(1)  # sentence
        b = refm.group(2)  # punctuation
        c = refm.group(3)  # reference
        # print("a:" + a + "\n b:" + b + "\n c:" + c + "\n")
        abbrevm = re.search(abbrevs, a)
        if abbrevm:
            swapped_string = swapped_string + a  # don't change order
            swapped_string = swapped_string + b
            swapped_string = swapped_string + c
        else:
            # for a I use match instead of search because match can search through newline etc. Whereas for c,
            # I am looking for a number at the beginning of a sentence, so it does not matter.
            if re.search('[0-9]]$', a) and re.search('^[1-9]]', c) and b == ".":
                # printl("a ends with number and c starts with number, in character by character comparison")
                swapped_string = swapped_string + a  # don't change order
                swapped_string = swapped_string + b
                swapped_string = swapped_string + c
            else:
                # print("order swapping")
                swapped_string = swapped_string + a  # sentence
                swapped_string = swapped_string + c  # reference or bracket
                swapped_string = swapped_string + b + " "  # punctuation
        ref_last = refm
    if ref_last:
        end_bit = clear_string[ref_last.end():]
    else:
        end_bit = clear_string[len(swapped_string):]
    # swapped_string = re.sub("</xref><xref", "</xref>, <xref", swapped_string)
    # print('swapped string')
    # print(swapped_string)
    # print('endbit')
    swapped_string = swapped_string + end_bit
    clear_string = swapped_string
    clear_string = re.sub(">\\.<", ">. <", clear_string)
    # print('\n\n\n Clear String:\n')
    # print(clear_string)
    sentence_pattern = "(" + sentenceFigLookbehind + "(" + secLookBehind + title + ")+" + ")|" + \
                       "(" + sentenceFigLookbehind + "(" + boxed + secLookBehind + title + "?)" + optEndTags1 + ")|" + \
                       "(" + sentenceFigLookbehind + wholeHeader + headerLookahead + ")|" + \
                       "(" + sentenceFigLookbehind + "(((" + tableWrap + ")+(" + Figure + ")+)|((" + Figure + ")+(" + \
                       tableWrap + ")+)|(" + tableWrap + ")+|(" + Figure + ")+)" + optEndTags + ")|" \
                       + "(" + sentenceFigLookbehind + "(((" + supplimentbehind + ")+" + optEndTags + ")|((" + \
                       supplimentbehind + ")+" + secLookBehind + title + ")))|" \
                       + "(" + sentenceFigLookbehind + "(" + refList + ")+" + optEndTags1 + ")|" \
                       + "(((" + sentenceFigLookbehind + "" + startSentence + \
                       ")|" + beginFirstSentence + "|" + beginPara + ")" + \
                       "(.*?)(Fig(s)?\\..+?)*?" + \
                       "(((?<!(" + endEquation + " ?|" + puncNoAbbrv + " ?|" + endPara + " ?))" \
                       + "(?=(?:" + nocapsParaLookAhead + ")))|" + \
                       "((?>" + endEquation + " ?|" + puncNoAbbrv + " ?|" + endPara + " ?)" \
                       + optCloseQuote + optReferenceSCI + manyEndTags + " ?" \
                       + "(?=(?:" + noSpaceReqLookahead + "|" + nocapsParaLookAhead + "|\n| *$)))|" + endLastSentence \
                       + "))"
    somethingFound = 0
    sentences = []
    # print('before finding sentences\n')
    for m in re.finditer(sentence_pattern, clear_string):
        somethingFound = 1
        # print('M TUPLE:')
        # print(m.group(0))
        # print('\n\n')
        sentences.append(m.group(0))
    if somethingFound == 0:
        print('No Sentence found')
    # else:
    #    print(sentences)
    refSentenceRev = re.compile("(.*?)" + atLeastOneRefSCI + capturePunctuation + "(\\s?(?:</p>)?)\\Z")
    count = 0
    newSentences = []
    for s in sentences:
        # print("in sentences: " + str(count) + ": " + s + "\n")
        refmRev = re.match(refSentenceRev, s)
        # print("refSentenceRev:"+refSentenceRev)
        # if sentence finishes with reference + punctuation, swap the two over
        if refmRev:
            # print("ref found")
            a = refmRev.group(1)  # sentence
            b = refmRev.group(2)  # reference or bracket or both
            c = refmRev.group(3)  # punctuation
            d = refmRev.group(4)  # space and/or </P>
            # print("sentence " + str(count) + ": " + s+ "A: " + a + "\n B: " + b + "\n C: "+ c +"\n", s, a, b, c)
            ns = ""
            # if (b.equals(")")) {
            if b.endswith(
                    ")"):  # { // made it endsWith instead of equals on 28/03/11 coz if reference is within bracket we do not want to swap punctuation, it worked, 31,39). is no longer becoming 31,.39) , why square bracket is not considered in bracket?????
                # #println("b ends with )")
                ns = ns + a  # sentence
                ns = ns + b  # bracket
                ns = ns + c  # punctuation
                ns = ns + d  # space
            else:
                # print("b does not end with )")
                if not b.startswith("<"):
                    index = b.index("<")
                    # print("index:"+index)
                    b1 = b[0:index]  # end index exclusive
                    # print("B1:"+b1)
                    # print("b at 0th position"+b.toCharArray()[0])
                    b2 = b[:index]
                    a = a + b1
                    b = b2
                ns = ns + a  # sentence
                ns = ns + c  # punctuation
                ns = ns + b  # reference
                ns = ns + d  # space
            # print("sentence " + str(count) + ": " + s + "A: " + a + "\n B: " + b + "\n C: " + c + "\n", s, a, b, c)
            newSentences.append(ns)
            # print("Sentence found: " + str(count))
        else:
            # print("sentence added s:"+s)
            if s.replace(" +", "") != "":
                newSentences.append(s)
        # print("not if nor else : count="+count)
        count = count + 1
    sentences = newSentences
    # Post-processing sentence array to move XML tags outside the sentences
    # ppSentenceTag = "(<s sid=\"\\d+\">)"
    ppStartTags = "((?:<abstract>|<body>|<abstracttext(?:.+?)>|<sec(?:.+?)>|<div(\\sDEPTH=\"\\d+\")?>|<p>|<p.+>|<disp-quote>|<list.+?>|<list-item>|<abstract.+?>|<statement>|<boxed-text.+?>|<boxed-text.+?><graphic.+?>)*)"
    # added overall brackets and ?: 15/6/09 mal //shs <sec> <abstract.+?>
    ppSentence = "(.+?)"
    ppEndTags = "((?:</abstract>|</body>|</abstract-title>|</sec>|</div>|</p>|</disp-quote>|</list>|</list-item>|<boxed-text.+?><graphic.+?></graphic></boxed-text>|</graphic>|</boxed-text>|</statement>|<list.+?>|<list-item>)*\\s?)\\Z"
    pp = re.compile(ppStartTags + ppSentence + ppEndTags)
    ppHeader = re.compile(".*?<caption.+?", flags=re.I)
    ppTitle = re.compile(".*?<title.*?>|.*?<title>", flags=re.I)
    ppFig = re.compile("<fig\\s?.+?>|<table-wrap.+?>", flags=re.I)
    # ppCap = re.compile("</caption.+?>|</fn.+?>",Pattern.CASE_INSENSITIVE); //shs comment : seems it can't find the tag without .+?
    ppCap = re.compile("</caption.+?>|</fn.+?>", flags=re.I)
    paraAttr = re.compile("<p.+></p>|<p>", flags=re.I)
    suppli = re.compile("<supplementary-material.+?>", flags=re.I)
    # named = re.compile("<named-content.+?>", flags=re.I)
    # boxed_text = re.compile("<boxed-text.+?>", flags=re.I)
    sent_id = sid
    # Vector<Element> allSentences = new Vector<Element>();
    # print("before sentences loop.")
    nsentences = []
    # print('before filling nsentences')
    for s in sentences:
        # print(s)
        ppm = re.search(pp, s)
        if ppm:
            # print("Sentence " + str(sent_id) + " matches the post- processing tags")
            one = ""
            if ppm.group(1):
                one = ppm.group(1)
            two = ppm.group(3)
            three = ""
            if ppm.group(4):
                three = ppm.group(4)
            # print("Group \nSent id is: " + str(sent_id) + " Group 1 is: " + ppm.group(1) + " Group 2 is: " + ppm.group(2) + "Group 3 is: " +ppm.group(3) + "Group 4 is: " + ppm.group(4))
            # print("one:" + str(one) + ", two: " + str(two) + ", three: " + str(three))
            mHead = re.search(ppHeader, s)
            mTitle = re.search(ppTitle, s)
            mfig = re.search(ppFig, s)
            mCap = re.search(ppCap, s)
            mParaAttr = re.search(paraAttr, s)
            mSuppli = re.search(suppli, s)
            # mNamed = re.search(named, s)
            # mBoxed = re.search(boxed_text, s)
            # s.insertChild(titleNodes.get(titleNodes.size()-1).getValue(), 0);
            if not (mHead or mTitle or mfig or mCap or mParaAttr or mSuppli):
                # print('head, title, mfig etc')
                # print(one + "<sent id=\"" + str(sent_id) + "\">" + two + "</sent>" + three)
                nsentences.append(
                    str(one) + "<SENT id=\"" + str(sent_id) + "\"><plain>" + str(two) + "</plain></SENT>" + str(three))
                # print("Sentence " + str(sent_id) + ": " + s)
                # print("after s tag:"+one + "<s sid=\"" + id+ "\">" + two + "</s>" + three);
                # if re.search(ppFig, two):
                #    print("two contains fig")
                # print(two)
                sent_id = sent_id + 1
            else:
                # this is a header, mHead matches
                # print("\n\n\nHeader : "+s+"\n\n ")
                '''
                if mHead:
                    # print("after s tag:" + one + "<sent id=\"" + str(sent_id) + "\">" + two + "</sent>" + three)
                    print("Head matched.")
                if mTitle:
                    # print("after s tag:" + one + "<sent id=\"" + str(sent_id) + "\">" + two + "</sent>" + three)
                    print("Title matched.")
                if mfig:
                    # print("after s tag:" + one + "<sent id=\"" + str(sent_id) + "\">" + two + "</sent>" + three)
                    print("ppFig matched.")
                if mCap:
                    # print("after s tag:" + one + "<sent id=\"" + str(sent_id) + "\">" + two + "</sent>" + three)
                    print("mCap matched.")
                if mParaAttr:
                    # print("after s tag:" + one + "<sent id=\"" + str(sent_id) + "\">" + two + "</sent>" + three)
                    print("ParaAttr matched.")
                if mSuppli:
                    # print("after s tag:" + one + "<sent id=\"" + str(sent_id) + "\">" + two + "</sent>" + three)
                    print("mSuppli matched.")
                '''
                # print('No head, title, mfig etc')
                # print(s)
                nsentences.append(s)  # ("<SENT id=\"" + str(sent_id) + "\">" + two + "</SENT>")

        else:
            print("Sentence " + str(sent_id) + "-- " + str(s) + " -- didn't match!")
    # print('\n\n\n Clear String:\n')
    # print(clear_string)
    # print("N sentences")
    # print(nsentences)
    clear_string = "".join(nsentences)
    clear_string = re.sub("</xref><xref", "</xref>, <xref", clear_string)
    clear_string = re.sub("</SENT> <SENT ", "</SENT><SENT ", clear_string)
    return sent_id, clear_string


# return StringInt(finalbuffer,sent_id)
def create_tag(soup, sid):
    sent_tag = soup.new_tag('SENT')
    sent_tag['id'] = sid
    return sent_tag


def reference_sents(ref_list, sent_id):
    # references are special sentences.
    # print('in ref_list sentence')
    n_sentences = ""
    for ch in ref_list.children:
        ch_str = str(ch)
        if ch.name == 'ref':
            sub_text = ''
            for gch in ch.children:
                # if gch.name in ['citation-alternatives', 'element-citation', 'mixed-citation', 'nlm-citation',
                # 'note', 'x']:
                sub_text = sub_text + " " + " ".join([d.string for d in gch.descendants if d.string])
                # print('sub_text')
                # print(sub_text)
            sent_id, n_sentences_method = sentence_split(sub_text, sent_id)
            n_sentences = n_sentences + n_sentences_method
        elif ch.name in ["sec", "fig", "statement", "div", "boxed-text", "list", "list-item", "disp-quote", "speech",
                         "fn-group", "fn", "def-list", "def-item", "def", "ack", "array", "table-wrap", "table",
                         "tbody", "caption", "answer", "sec-meta", "glossary", "question", "question-wrap", "x"]:
            # print("in first elseif")
            sent_id, n_sentences_method = call_sentence_tags(ch, sent_id)
            n_sentences = n_sentences + ch_str[:ch_str.find('>') + 1] + n_sentences_method + "</" + gch.name + ">"
        else:
            n_sentences = n_sentences + ch_str
    return sent_id, n_sentences


def process_p_tag(gch, sent_id):
    n_sentences = ""
    p_children = gch.contents
    gch_str = str(gch)
    if len(p_children) == 1 and (not p_children[0].string) and (p_children[0].name in ["ext-link", "e-mail", "uri", "inline-supplementary-material",
                                           "related-article", "related-object", "address", "alternatives", "array",
                                           "funding-source", "inline-graphic"]):
        n_sentences = n_sentences + gch_str
    else:
        # print('P tag:\n')
        # print(p_tag)
        # print(p_tag[3:-4])
        # check if p contains graphic and boxed-text without text inside.
        sent_id, n_sentences_method = sentence_split(gch_str[gch_str.find('>')+1:-4], sent_id)
        # print(n_sentences_method)
        # print(sub_sent)
        n_sentences = n_sentences + gch_str[:gch_str.find('>') + 1] + n_sentences_method + "</" + gch.name + ">"
    return sent_id, n_sentences


def call_sentence_tags(ch, sent_id):
    #print('In call_sentence_tags')
    # print()
    n_sentences = ""
    for gch in ch.children:
        # print(gch.name)
        gch_str = str(gch)
        if gch.name in ['article-title', 'title', 'subtitle', 'trans-title', 'trans-subtitle', 'alt-title', 'label', 'td', 'th']:
            # assumption is that these tags only contain one sentence.
            # print("GCH")
            # print(str(gch))
            if gch.find('p', recursive=False):
                sent_id, n_sentences_method = call_sentence_tags(gch, sent_id)
                n_sentences = n_sentences + gch_str[:gch_str.find('>') + 1] + n_sentences_method + "</" + gch.name + ">"
            else:
                sub_text = [str(d) for d in gch.children]
                # print(sub_text)
                n_sentences = n_sentences + gch_str[:gch_str.find('>') + 1] + '<SENT id="' + str(
                    sent_id) + '"><plain>' + "".join(sub_text) + "</plain></SENT></" + gch.name + ">"
                sent_id = sent_id + 1
        elif gch.name in ["sec", "fig", "statement", "div", "boxed-text", "list", "list-item", "disp-quote", "speech",
                          "fn-group", "fn", "def-list", "def-item", "def", "ack", "array", "table-wrap", "table",
                          "tbody", "thead", "tr", "caption", "answer", "sec-meta", "glossary", "question", "question-wrap"]:
            # print("in first elseif : sec etc string")
            # print(gch_str)
            sent_id, n_sentences_method = call_sentence_tags(gch, sent_id)
            n_sentences = n_sentences + gch_str[:gch_str.find('>') + 1] + n_sentences_method + "</" + gch.name + ">"
            # print(gch_str)
            # print(gch_str[:gch_str.find('>')+1]) # works, tag.name will
            # print("n sentences method")
            # print(n_sentences_method)
            # print(gch_str[:gch_str.find('>')+1] + "".join(n_sentences_method) + "</" + gch.name + ">")
            # may need to do further changes
        elif gch.name == 'p':
            sent_id, n_sentences_method = process_p_tag(gch, sent_id)
            n_sentences = n_sentences + n_sentences_method
        else:
            # convert the xml into string and concat
            n_sentences = n_sentences + gch_str

    return sent_id, n_sentences


# def process_front(front):
#     #print('In process_front')
#     sent_id = 1
#     n_sentences = ""
#     if front.find('article-meta'):
#         # print('In article-meta')
#         art_meta = front.find('article-meta')
#         # print(art_meta)
#         # directly using 0th item because according to the JATS DTD it should appears precisely once.

#         for ch in art_meta.find_all(recursive=False):
#             ch_str = str(ch)
#             if ch.name in ['title-group', 'supplement', 'supplementary-material', 'abstract', 'trans-abstract',
#                            'kwd-group', 'funding-group']:
#                 sent_id, n_sentences_method = call_sentence_tags(ch, sent_id)
#                 n_sentences = n_sentences + ch_str[:ch_str.find('>') + 1] + n_sentences_method + "</" + ch.name + ">"
#             else:
#                 n_sentences = n_sentences + ch_str
#         #print('\n\nend of process_front')
#     return sent_id, n_sentences


# def process_body(body, sent_id):
#     n_sentences_parts = []
#     for ch in body.find_all(recursive=False):
#         if ch.name == 'p':
#             sent_id, n_sentences_method = process_p_tag(ch, sent_id)
#             n_sentences_parts.append(n_sentences_method)
#         elif ch.name in ['sec', 'ack', 'alternatives', 'array', 'preformat', 'fig', 'fig-group', 'question-wrap',
#                          'question-wrap-group', 'list', 'table-wrap-group', 'table-wrap', 'display-formula',
#                          'display-formula-group', 'def-list', 'list', 'supplementary-material', 'kwd-group',
#                          'funding-group', 'statement', 'fig']:
#             sent_id, n_sentences_method = call_sentence_tags(ch, sent_id)
#             tag_open = str(ch)[:str(ch).find('>') + 1]
#             tag_close = "</" + ch.name + ">"
#             n_sentences_parts.append(tag_open + n_sentences_method + tag_close)
#         else:
#             n_sentences_parts.append(str(ch))
    
#     n_sentences = ''.join(n_sentences_parts)
#     return sent_id, n_sentences

# def process_back(back, sent_id):
#     # sent_id = 0
#     #print('In process_back')
#     n_sentences = ""
#     for ch in back.find_all(recursive=False):
#         ch_str = str(ch)
#         if ch.name in ['sec', 'p', 'ack', 'alternatives', 'array', 'preformat', 'fig', 'fig-group' 'question-wrap',
#              'question-wrap-group', 'list', 'table-wrap-group', 'table-wrap', 'display-formula',
#              'display-formula-group', 'def-list', 'list', 'supplementary-material', 'kwd-group',
#              'funding-group', 'statement', 'ref-list', 'glossary']:
#             # print(ch.name)
#             if ch.name == 'ref-list':
#                 #print('\n\n\n in ref-list')
#                 sent_id, n_sentences_method = reference_sents(ch, sent_id)
#             else:
#                 sent_id, n_sentences_method = call_sentence_tags(ch, sent_id)
#             n_sentences = n_sentences + ch_str[:ch_str.find('>') + 1] + n_sentences_method + "</" + ch.name + ">"
#         else:
#             n_sentences = n_sentences + ch_str
#     #print('\n\nend of process_body')
#     return sent_id, n_sentences


# def process_full_text(each_file):  
#     # replacing body tag with orig_body to stop BeautifulSoup from removing this tag.
#     each_file = each_file.replace('<body>', '<orig_body>')
#     each_file = each_file.replace('<body ', '<orig_body ')
#     each_file = each_file.replace('</body>', '</orig_body>')
#     try:
#         xml_soup = BeautifulSoup(each_file, 'lxml')
#         # BeautifulSoup adds an extra html and body tag. removing those 2 tags.
#         if xml_soup.html:
#             # print('html unwrap')
#             xml_soup.html.unwrap()
#         if xml_soup.body:
#             # print('body unwrap')
#             xml_soup.body.unwrap()
#         # print(xml_soup.find('front'))
#         if xml_soup.find('orig_body'):
#             # print('orig_body find')
#             xml_soup.find('orig_body').name = 'body'
#         sent_id = 1
#         xml_string = None
#         # Access regions of interests for sentence splitting.
#         last_tag=None
#         if xml_soup.article.find('front'):
#             index = str(xml_soup).index('<front>')
#             # print("\n\nFront index: " + str(index))
#             xml_string = str(xml_soup)[:index] + "<front>"
#             # print("Pre-front")
#             # print(xml_string)
#             sent_id, n_sentences_front = process_front(xml_soup.article.find('front'))
#             xml_string = xml_string + n_sentences_front + "</front>"
#             last_tag = "</front>"
#         if xml_soup.article.find('body'):
#             if xml_string == None:
#                 index = str(xml_soup).index('<body>')
#                 # print("\n\nBody index: " + str(index))
#                 xml_string = str(xml_soup)[:index]
#                 # print("Pre-body")
#                 # print(xml_string)
#             sent_id, n_sentences_body = process_body(xml_soup.article.find('body'), sent_id)
#             xml_string = xml_string + "<body>" + n_sentences_body + "</body>"
#             last_tag = "</body>"
#         if xml_soup.article.find('back'):
#             if xml_string == None:
#                 index = str(xml_soup).index('<back>')
#                 # print("\n\nBack index: " + str(index))
#                 xml_string = str(xml_soup)[:index]
#                 # print("Pre-back")
#                 # print(xml_string)
#             sent_id, n_sentences_back = process_back(xml_soup.article.find('back'), sent_id)
#             xml_string = xml_string + "<back>" + n_sentences_back + "</back>"
#             '''    
#                 print('\n\n\nFront sentences:\n')
#                 print(n_sentences_front)
#                 print('\n\n\nBody sentences:\n')
#                 print(n_sentences_body)
#                 print('\n\n\nBack sentences:\n')
#                 print(n_sentences_back)
#             '''
#             last_tag = "</back>"
#         if last_tag:
#             index_last_tag = str(xml_soup).index(last_tag)
#             xml_string = xml_string + str(xml_soup)[index_last_tag + len(last_tag):]
#             return xml_string
#         else:
#             print("No FRONT, BODY BACK.")
#             xml_string = str(xml_soup)
#             return xml_string

#     except Exception as e:
#         print(e)
#         return ""  # Return an empty string or handle the exception appropriately

def process_body(body, sent_id):
    n_sentences_parts = []
    for ch in body.find_all(recursive=False):
        if ch.name == 'p':
            sent_id, n_sentences_method = process_p_tag(ch, sent_id)
            n_sentences_parts.append(n_sentences_method)
        elif ch.name in ['sec', 'ack', 'alternatives', 'array', 'preformat', 'fig', 'fig-group', 'question-wrap',
                         'question-wrap-group', 'list', 'table-wrap-group', 'table-wrap', 'display-formula',
                         'display-formula-group', 'def-list', 'list', 'supplementary-material', 'kwd-group',
                         'funding-group', 'statement', 'fig']:
            sent_id, n_sentences_method = call_sentence_tags(ch, sent_id)
            tag_open = str(ch)[:str(ch).find('>') + 1]
            tag_close = "</" + ch.name + ">"
            n_sentences_parts.append(tag_open + n_sentences_method + tag_close)
        else:
            n_sentences_parts.append(str(ch))
    
    n_sentences = ''.join(n_sentences_parts)
    return sent_id, n_sentences

def process_back(back, sent_id):
    n_sentences_parts = []
    for ch in back.find_all(recursive=False):
        if ch.name in ['sec', 'p', 'ack', 'alternatives', 'array', 'preformat', 'fig', 'fig-group', 'question-wrap',
                       'question-wrap-group', 'list', 'table-wrap-group', 'table-wrap', 'display-formula',
                       'display-formula-group', 'def-list', 'list', 'supplementary-material', 'kwd-group',
                       'funding-group', 'statement', 'ref-list', 'glossary']:
            if ch.name == 'ref-list':
                sent_id, n_sentences_method = reference_sents(ch, sent_id)
            else:
                sent_id, n_sentences_method = call_sentence_tags(ch, sent_id)
            tag_open = str(ch)[:str(ch).find('>') + 1]
            tag_close = "</" + ch.name + ">"
            n_sentences_parts.append(tag_open + n_sentences_method + tag_close)
        else:
            n_sentences_parts.append(str(ch))
    
    n_sentences = ''.join(n_sentences_parts)
    return sent_id, n_sentences

def process_front(front):
    sent_id = 1
    n_sentences_parts = []
    if front.find('article-meta'):
        art_meta = front.find('article-meta')
        for ch in art_meta.find_all(recursive=False):
            if ch.name in ['title-group', 'supplement', 'supplementary-material', 'abstract', 'trans-abstract',
                           'kwd-group', 'funding-group']:
                sent_id, n_sentences_method = call_sentence_tags(ch, sent_id)
                tag_open = str(ch)[:str(ch).find('>') + 1]
                tag_close = "</" + ch.name + ">"
                n_sentences_parts.append(tag_open + n_sentences_method + tag_close)
            else:
                n_sentences_parts.append(str(ch))
    
    n_sentences = ''.join(n_sentences_parts)
    return sent_id, n_sentences


def process_full_text(each_file):  
    # Replacing body tag with orig_body to prevent BeautifulSoup from removing this tag.
    each_file = each_file.replace('<body>', '<orig_body>')
    each_file = each_file.replace('<body ', '<orig_body ')
    each_file = each_file.replace('</body>', '</orig_body>')

    try:
        xml_soup = BeautifulSoup(each_file, 'lxml')

        # BeautifulSoup adds an extra html and body tag. Removing those tags.
        if xml_soup.html:
            xml_soup.html.unwrap()
        if xml_soup.body:
            xml_soup.body.unwrap()

        # Renaming 'orig_body' back to 'body'
        if xml_soup.find('orig_body'):
            xml_soup.find('orig_body').name = 'body'

        processed_parts = []
        sent_id = 1

        for part in ['front', 'body', 'back']:
            tag = xml_soup.article.find(part)
            if tag:
                if part == 'front':
                    sent_id, processed_content = process_front(tag)
                elif part == 'body':
                    sent_id, processed_content = process_body(tag, sent_id)
                elif part == 'back':
                    sent_id, processed_content = process_back(tag, sent_id)
                
                processed_parts.append(f'<{part}>{processed_content}</{part}>')
            else:
                processed_parts.append(f'<{part}></{part}>')  # Empty tag if not found

        return ''.join(processed_parts)

    except Exception as e:
        print(e)
        return ""  # Return an empty string or handle the exception appropriately

    
    
def process_abstract_text(gch, sent_id, gch_flag):
    n_sentences = ""
    p_children = gch.contents
    gch_str = str(gch)
    sent_id, n_sentences_method = sentence_split(gch_str[gch_str.find('>') + 1:-(len("</" + gch.name + ">"))], sent_id)
    #print("n sentences in process anbstract text")
    #print(n_sentences_method)
    # print(sub_sent)
    if gch_flag==1:
        n_sentences = n_sentences + gch_str[:gch_str.find('>') + 1] + n_sentences_method + "</" + gch.name + ">"
    else:
        n_sentences = n_sentences + n_sentences_method
    return sent_id, n_sentences


def process_abstracts(each_file, out_file):
    sent_id = 1
    try:
        xml_soup = BeautifulSoup(each_file, 'lxml')
        # BeautifulSoup adds an extra html and body tag. removing those 2 tags.
        if xml_soup.html:
            # print('html unwrap')
            xml_soup.html.unwrap()
        if xml_soup.body:
            # print('body unwrap')
            xml_soup.body.unwrap()
        n_sentences = ""
        #print(str(xml_soup))
        target_tags = xml_soup.article.find_all(["abstract"])
        if len(target_tags)>0:
            # abstract_tags = xml_soup.pubmedarticle.medlinecitation.find_all(["abstract", "otherabstract"], recursive=False)
            prev = None
            xml_str = str(xml_soup)
            #print(target_tags)
            if target_tags:
                # print("targets found")
                # print(abstract_tags)
                ch_tag_end = 0
                prev_start_index=0
                prev=None
                for ch in target_tags:
                    #print(ch.name)
                    ch_str = str(ch)
                    match = re.search("<" + ch.name +"[^>]*?>", xml_str)
                    if match:
                        #print(match.group())
                        ch_tag_end = match.end()
                        #print("ch_tag_end:" + str(ch_tag_end))
                    if prev==None:
                        n_sentences = xml_str[:ch_tag_end]
                        prev = ch
                    else:
                        # find end of previous ch
                        #print("previous name:" + prev.name)
                        match = re.search("</"+prev.name+">", xml_str)
                        if match:
                            prev_start_index = match.start()
                        # print("prev_start:" + str(prev_start_index))
                        # print(match)
                        n_sentences = n_sentences + xml_str[prev_start_index:ch_tag_end]
                        prev=ch
                    # print("before gchildren loop")
                    # print(n_sentences)
                    # print("\nEND of n_sentences")
                    if ch.name == "title":
                        #print("article title")
                        sent_id, n_sentences_method = process_abstract_text(ch, sent_id, 0)
                        #print(n_sentences_method)
                        n_sentences = n_sentences + n_sentences_method
                    else:
                        for gch in ch.find_all(recursive=False):
                            gch_str = str(gch)
                            if gch.name=='title' or gch.name=='p':
                                #print("abstracttext")
                                sent_id, n_sentences_method = process_abstract_text(gch, sent_id, 1)
                                #print(n_sentences_method)
                                n_sentences = n_sentences + n_sentences_method
                            else:
                                n_sentences = n_sentences + gch_str
                if prev:
                    match = re.search("</"+prev.name+">", xml_str)
                    if match:
                        prev_start_index = match.start()
                    n_sentences = n_sentences + xml_str[prev_start_index:]
                else:
                    print("\nERROR: Prev should not be none")
            else:
                print("No Abstract or Other Abstract\n")
                n_sentences=xml_str
        with open(out_file, 'a') as out:
            out.write(n_sentences)
    except Exception as e:
        print(e)
    

def process_each_article(xml_content, document_flag):
    files_list = get_blocks_from_xml_string(xml_content, document_flag)
    processed_content = ""
    
    if document_flag == 'a':
        processed_content += "<articles>\n"
    
    for each_file in tqdm(files_list):
        if document_flag == 'f':
            processed_content += process_full_text(each_file)
        elif document_flag == 'a':
            print("abstract")
            processed_content += process_abstracts(each_file)
        else:
            print("Wrong document Flag")
            return ""

    if document_flag == 'a':
        processed_content += "</articles>\n"

    return processed_content

# # Example usage
# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(
#         description='This script will process XML articles in JAT-Archive DTD and do sentence splitting')
#     parser.add_argument("-x", "--xml", required=True, help="XML content as string", metavar="XML_STRING")
#     parser.add_argument("-d", "--document", required=True, help="Document type flag", metavar="FLAG")
#     args = parser.parse_args()

#     modified_xml = process_each_article(args.xml, args.document)
#     print(modified_xml)
#     print("Sentenciser Finished!")


In [28]:
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm
import random
import sys, io, re, os
import argparse
from collections import defaultdict
import re


titleMapsBody = {
    'INTRO': ['introduction', 'background', 'related literature', 'literature review', 'objective', 'aim ', 'purpose of this study', 'study (purpose|aim|aims)', '(\d)+\. (purpose|aims|aim)', '(aims|aim|purpose) of the study', '(the|drug|systematic|book) review', 'review of literature', 'related work', 'recent advance'],
    'METHODS': ['supplement', 'methods and materials', 'method', 'material', 'experimental procedure', 'implementation', 'methodology', 'treatment', 'statistical analysis', "experimental", '(\d)+\. experimental$', 'experimental (section|evaluation|design|approach|protocol|setting|set up|investigation|detail|part|pespective|tool)', "the study", '(\d)+\. the study$', "protocol", "protocols", 'study protocol', 'construction and content', 'experiment (\d)+', '^experiments$', 'analysis', 'utility', 'design', '(\d)+\. theory$', "theory", 'theory and ', 'theory of '],
    'RESULTS': ['result', 'finding', 'diagnosis'],
    'DISCUSS': ['discussion', 'management of', '(\d)+\. management', 'safety and tolerability', 'limitations', 'perspective', 'commentary', '(\d)+\. comment'],
    'CONCL': ['conclusion', 'key message', 'future', 'summary', 'recommendation', 'implications for clinical practice','concluding remark'],
    'CASE': ['case study report', 'case report', 'case presentation', 'case description', 'case (\d)+', '(\d)+\. case', 'case summary', 'case history'],
    'ACK_FUND': ['funding', 'acknowledgement', 'acknowledgment', 'financial disclosure'],
    'AUTH_CONT': ['author contribution', 'authors\' contribution', 'author\'s contribution'],
    'COMP_INT': ['competing interest', 'conflict of interest', 'conflicts of interest', 'disclosure', 'decleration'],
    'ABBR': ['abbreviation'],
    'SUPPL': ['supplemental data', 'supplementary file', 'supplemental file', 'supplementary data', 'supplementary figure', 'supplemental figure', 'supporting information', 'supplemental file', 'supplemental material', 'supplementary material', 'supplement material', 'additional data files', 'supplemental information', 'supplementary information', 'supplemental information', 'supporting information', 'supplemental table', 'supplementary table', 'supplement table', 'supplementary material', 'supplemental material', 'supplement material', 'supplementary video']
}

titleExactMapsBody = {
    'INTRO': ["aim", "aims", "purpose", "purposes", "purpose/aim", "purpose of study", "review", "reviews", "minireview"],
    'METHODS': ["experimental", "the study", "protocol", "protocols"],
    'DISCUSS': ["management", "comment", "comments"],
    'CASE': ["case", "cases"]
}

titleMapsBack = {
    'REF': ['reference', 'literature cited', 'references', 'bibliography'],
    'ACK_FUND': ['funding', 'acknowledgement', 'acknowledgment', 'aknowledgement', 'acknowlegement', 'open access', 'financial support', 'grant', 'author note', 'financial disclosure'],
    'ABBR': ['abbreviation', 'glossary'],
    'COMP_INT': ['competing interest', 'conflict of interest', 'conflicts of interest', 'disclosure', 'decleration', 'conflits', 'interest'],
    'SUPPL': ['supplementary', 'supporting information', 'supplemental', 'web extra material'],
    'APPENDIX': ['appendix', 'appendices'],
    'AUTH_CONT': ['author', 'contribution']
}



def categorize_section(title):
    # Lowercase the title for consistency
    title_lower = title.lower()

    # INTRO
    if any(re.search(pattern, title_lower) for pattern in ['introduction', 'background', 'literature review', 'objective', 'purpose of this study', 'systematic review', 'recent advance']):
        return 'INTRO'

    # METHODS
    # Enhanced with NLP logic: If the title contains 'experiment' or 'study', categorize as METHODS
    if 'experiment' in title_lower or 'study' in title_lower:
        return 'METHODS'
    if any(re.search(pattern, title_lower) for pattern in ['methods and materials', 'methodology', 'experimental procedure', 'implementation', 'treatment', 'statistical analysis', 'experimental design', 'study protocol', 'design theory']):
        return 'METHODS'

    # RESULTS
    if any(re.search(pattern, title_lower) for pattern in ['result', 'finding', 'diagnosis']):
        return 'RESULTS'

    # DISCUSS
    if any(re.search(pattern, title_lower) for pattern in ['discussion', 'management of', 'safety and tolerability', 'limitations', 'perspective', 'commentary']):
        return 'DISCUSS'

    # CONCL
    if any(re.search(pattern, title_lower) for pattern in ['conclusion', 'key message', 'future direction', 'summary', 'recommendation', 'implications for clinical practice', 'concluding remark']):
        return 'CONCL'

    # CASE
    if any(re.search(pattern, title_lower) for pattern in ['case study report', 'case presentation', 'case description', 'case history']):
        return 'CASE'

    # ACK_FUND
    if any(re.search(pattern, title_lower) for pattern in ['funding', 'acknowledgement', 'financial disclosure']):
        return 'ACK_FUND'

    # AUTH_CONT
    if any(re.search(pattern, title_lower) for pattern in ['author contribution']):
        return 'AUTH_CONT'

    # COMP_INT
    if any(re.search(pattern, title_lower) for pattern in ['competing interest', 'conflict of interest', 'disclosure']):
        return 'COMP_INT'

    # ABBR
    if any(re.search(pattern, title_lower) for pattern in ['abbreviation', 'glossary']):
        return 'ABBR'

    # SUPPL
    if any(re.search(pattern, title_lower) for pattern in ['supplemental data', 'supplementary material', 'additional data files', 'supporting information']):
        return 'SUPPL'

    # If no category matches, return 'OTHER'
    return 'OTHER'

# Assuming soup is a BeautifulSoup object of the parsed XML/HTML document

def extract_sections_and_sentences(soup, titleMapsBody, titleExactMapsBody, titleMapsBack):
    sections_sentences = defaultdict(list)
    
    section_order = []
    
    # Function to identify section type based on title
    def identify_section_type(title, secFlag):
        title_lower = title.lower()

        if secFlag == 'body':
            title_map = titleMapsBody
            exact_title_map = titleExactMapsBody
        elif secFlag == 'back':
            title_map = titleMapsBack
            exact_title_map = None
        else:
            return 'OTHER'

        # Check for exact match first
        if exact_title_map:
            for key, patterns in exact_title_map.items():
                if title_lower in patterns:
                    return key

        # Check for partial match
        for key, patterns in title_map.items():
            if any(re.search(pattern, title_lower) for pattern in patterns):
                return key

        # Default to 'OTHER' if no match found
        return 'OTHER'

     # Function to extract and categorize sections
    def extract_and_categorize_sections(soup_section, secFlag):
        for sec in soup_section.find_all('sec', recursive=False):
            if sec.title:
                section_type = identify_section_type(sec.title.text, secFlag)
                section_order.append(section_type)
                for p in sec.find_all('p', recursive=False):
                    sections_sentences[section_type].append(p.text.strip())
                    
    # Extract from body and back sections
    if soup.body:
        extract_and_categorize_sections(soup.body, 'body')
    if soup.back:
        extract_and_categorize_sections(soup.back, 'back')

    # Apply logic for 'OTHER' sections
    for index, section in enumerate(section_order):
        if section == 'OTHER':
            # If it follows 'INTRO', classify as 'METHODS'
            if index > 0 and section_order[index - 1] == 'INTRO':
                sections_sentences['METHODS'].extend(sections_sentences['OTHER'])
                section_order[index] = 'METHODS'
            # If it precedes 'DISCUSS', classify as 'RESULTS'
            elif index < len(section_order) - 1 and section_order[index + 1] == 'DISCUSS':
                sections_sentences['RESULTS'].extend(sections_sentences['OTHER'])
                section_order[index] = 'RESULTS'

    # Remove the reclassified 'OTHER' sections
    if 'OTHER' in sections_sentences:
        del sections_sentences['OTHER']

    return sections_sentences



def process_each_file(filename):
    each_file = each_file.replace('<body>', '<orig_body>')
    each_file = each_file.replace('<body ', '<orig_body ')
    each_file = each_file.replace('</body>', '</orig_body>')
    xml_soup = BeautifulSoup(each_file, 'lxml')
    xml_soup.html.unwrap()
    xml_soup.body.unwrap()
    if xml_soup.find('orig_body'):
        #print("orig_body found")
        xml_soup.find('orig_body').name = 'body'

    extract_sections_and_sentences(xml_soup)


def extract_section_titles(xml_content):
    # Parse the XML/HTML content
    # Find all 'sec' tags and extract their 'title' tags
    section_titles = [sec.find('title').get_text(strip=True) for sec in soup.find_all('sec') if sec.find('title')]


    return section_titles


In [32]:
import requests
# pmcid = 'PMC8080698'

pmcid = 'PMC1501050'

url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
response = requests.get(url)

if response.status_code == 200:
    xml_content = response.content.decode('utf-8')
#     processed_xml = process_each_article(xml_content, document_flag='f')
    processed_xml = get_blocks_from_xml_string(xml_content, document_flag='f')
#     soup = BeautifulSoup(response.content, 'xml')
#     p_tags = soup.find_all('p')
#     p_texts = [tag.get_text() for tag in p_tags]
#     # Using nlp.pipe for batch processing
#     for doc in nlp.pipe(p_texts):
#         sentences = [sent.text.strip() for sent in doc.sents]
#         segmented_sentences.extend(sentences)

In [33]:
soup = BeautifulSoup(processed_xml[0], 'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<article article-type="product-review" xmlns:xlink="http://www.w3.org/1999/xlink"><front><journal-meta><journal-id journal-id-type="nlm-ta">BMC Bioinformatics</journal-id><journal-title>BMC Bioinformatics</journal-title><issn pub-type="epub">1471-2105</issn><publisher><publisher-name>BioMed Central</publisher-name><publisher-loc>London</publisher-loc></publisher></journal-meta><article-meta><article-id pub-id-type="pmcid">1501050</article-id><article-id pub-id-type="publisher-id">1471-2105-7-194</article-id><article-id pub-id-type="pmid">16600052</article-id><article-id pub-id-type="doi">10.1186/1471-2105-7-194</article-id><article-categories><subj-group subj-group-type="heading"><subject>Software</subject></subj-group></article-categories><title-group><article-title>Mining gene expression data by interpreting principal components</article-title></title-group><contrib-group><contrib contrib-type="author" corresp="yes" id="A1"><name><surname>Roden<

In [34]:
dd = extract_section_titles(soup)
dd

['Background',
 'Results',
 'Conclusion',
 'Background',
 'Implementation',
 'Results',
 'Application to microarray expression data, Case 1: GNF human data',
 'Application to microarray expression data, case 2: human diabetes',
 'Discussion',
 'Conclusion',
 'Availability and requirements',
 'Methodology',
 'Determine the principal components of the dataset',
 'Identify extreme gene probes for each principal component',
 'Identify significant conditions for each principal component',
 'Interpret each principal component using covariate annotations',
 'Terminating condition',
 "Authors' contributions",
 'Supplementary Material',
 'Acknowledgements',
 'Figures and Tables']

In [29]:
dd = extract_section_titles(soup)
dd

['Introduction',
 'Structural heterogeneity related to the function of PcG proteins',
 'The role of PcG proteins in immune regulation',
 'Current RNA-seq and chromatin immunoprecipitation (ChIP)-seq data for identifying the target loci of PcG proteins',
 'Therapeutic agents for treating hematopoietic malignancies by inhibiting the activity of PcG proteins',
 'Conclusion and future prospects']

In [92]:
sent_xml = process_each_article(processed_xml[0], document_flag='f')

100%|█████████████████████████████████████████████| 1/1 [00:45<00:00, 45.05s/it]


In [23]:
dd.keys()

dict_keys(['INTRO', 'CONCL', 'METHODS'])